In [3]:
!pip3 install chromadb
!pip3 install langchain "unstructured[pdf]" sentence_transformers

7349.07s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


  Obtaining dependency information for chromadb from https://files.pythonhosted.org/packages/f1/2a/549be867b5ab45112aacd9d113af788768a015e9e6d0ade831b45c1df877/chromadb-0.4.15-py3-none-any.whl.metadata
  Obtaining dependency information for requests>=2.28 from https://files.pythonhosted.org/packages/70/8e/0e2d847013cb52cd35b38c009bb167a1a26b2ce6cd6965bf26b47bc0bf44/requests-2.31.0-py3-none-any.whl.metadata
  Obtaining dependency information for pydantic>=1.9 from https://files.pythonhosted.org/packages/73/66/0a72c9fcde42e5650c8d8d5c5c1873b9a3893018020c77ca8eb62708b923/pydantic-2.4.2-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.6/158.6 kB 1.2 MB/s eta 0:00:00a 0:00:01
  Obtaining dependency information for chroma-hnswlib==0.7.3 from https://files.pythonhosted.org/packages/94/3f/844393b0d2ea1072b7704d6eff5c595e05ae8b831b96340cdb76b2fe995c/chroma_hnswlib-0.7.3-cp311-cp311-macosx_10_9_x86_64.whl.metadata
  Obtaining dependency information for fastapi>=0.95.2 f

7404.90s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


  Obtaining dependency information for langchain from https://files.pythonhosted.org/packages/ad/b1/6bb5006471264b5d75fcf0e3d7ed8d0bfc4ec335e08e05abf5900c42aa43/langchain-0.0.325-py3-none-any.whl.metadata
  Obtaining dependency information for unstructured[pdf] from https://files.pythonhosted.org/packages/cc/64/89317032efe3f499c62cfa7a9dca1c39a0e379c4712573e045f8aad7f0eb/unstructured-0.10.27-py3-none-any.whl.metadata
  Using cached sentence-transformers-2.2.2.tar.gz (85 kB)
  Preparing metadata (setup.py) ... done
  Obtaining dependency information for SQLAlchemy<3,>=1.4 from https://files.pythonhosted.org/packages/5a/a6/95f984d3cfe22e0e8a71458468ad362a8869be7ec434d78adf4347602d5c/SQLAlchemy-2.0.22-cp311-cp311-macosx_10_9_x86_64.whl.metadata
  Obtaining dependency information for aiohttp<4.0.0,>=3.8.3 from https://files.pythonhosted.org/packages/0b/f7/6fce2fa9c9352c2b0a9868cbd6aaab57a46fa52330e90c0fa0cdf990cec0/aiohttp-3.8.6-cp311-cp311-macosx_10_9_x86_64.whl.metadata
  Obtaining depen

### Load the Documents

Here we use Langchain to load all documents from our `./docs` directory. More info on how this works can be found on [Langhchain's website](https://python.langchain.com/docs/modules/data_connection/document_loaders/file_directory)

In [4]:
from langchain.document_loaders import DirectoryLoader
directory = './docs'

def load_docs(directory):
  loader = DirectoryLoader(directory)
  documents = loader.load()
  return documents

documents = load_docs(directory)
len(documents)

2

### Chunking the Documents

Here we chunk the document's contents from which embeddings will be generated. You can experiment with the `chunk_size` and `chunk_overlap` to fit your use case.

In [5]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

def split_docs(documents,chunk_size=1000,chunk_overlap=100):
  text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
  docs = text_splitter.split_documents(documents)
  return docs

docs = split_docs(documents)
print(len(docs))

51


### Choose Your Embedding Model

Here we're using `BAAI/bge-small-en` as our model, but this can be swapped out with any one of the BGE models from the [BGE Model List](https://huggingface.co/BAAI/bge-large-en)

In [21]:
# Example of using Setence Transformers instead of BGE models
# from langchain.embeddings import SentenceTransformerEmbeddings
# embeddings = SentenceTransformerEmbeddings(model_name="instructor-large")

from langchain.embeddings import HuggingFaceBgeEmbeddings

model_name = "BAAI/bge-small-en"
model_kwargs = {"device": "cpu"}
encode_kwargs = {"normalize_embeddings": True}
embeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs
)

### Load Vectors into Chroma DB

Here we load the vector embeddings from our documents into the [Chroma DB](https://docs.trychroma.com/getting-started) vector store. Note this could easily be swapped out with something like Pinecone for higher throughput.

In [22]:
from langchain.vectorstores import Chroma
db = Chroma.from_documents(docs, embeddings)

len(db.get()['ids'])




153

### Query the Vector DB

Here we query the locally running instance of Chroma DB. By default we want to return the top 3 results. Increase `k=3` if you would like to return more than 3 of the top results.

In [30]:
query = "What is the total cost of the Hubsync project?"
matching_docs = db.similarity_search(query,k=3)

matching_docs

[Document(page_content='Telios will maintain open communication with HubSync throughout the engagement and provide\n\nsupport for any inquiries related to the project.\n\nCosting\n\nPhase I: $40,000.00\n\nPhase II: $250 / hour T&M\n\nEngagement Details\n\nPhase I: 50% due upon execution of SOW, 30% upon delivery of LLM, 20% upon system\n\nacceptance. Invoices are due upon receipt.\n\nPhase II: T&M hours shall be invoiced monthly. Invoices are due net-30.\n\nAll services are to be provided remotely unless advance request for on-site.\n\nContact(s)\n\nGareth Harte Founder & CEO/CTO (484) 753-5896 gareth@corp.telios.io\n\nCamden Weis Chief Revenue Officer (336) 491-9517 camden@corp.telios.io\n\nAcceptance:\n\nThis Statement of Work, as described above, has been accepted by HubSync and Telios.\n\nHubSync Co.\n\n_______________________________ Signature\n\n_______________________________ Name\n\n________________________________ Title\n\n________________________________ Date\n\nTelios Techno